In [1]:
import pickle
import numpy as np
import pandas as pd
import os
import time
import torch

# Import custom functionalities 
from config import MODEL_PATH, DATA_PATH, find_device
from data_prep import load_and_preprocess_data, split_data, general_revenue_dataframe, split_univariate_data, arrays_to_tensors, get_lag_llama_dataset
from features import add_calendar_features, add_sales_proxies,  full_data_target_encoder
from predictions import retrieve_predictions, lstm_model_predictions, get_lag_llama_predictions, transfrom_lag_llama_predictions
from models import sliding_windows, LSTM, LSTM2
from evaluate import evaluate_model

Using MPS device


/Users/nairabarseghyan/anaconda3/lib/python3.11/site-packages/google/colab/data_table.py:30: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


Using MPS device


In [17]:
seq_length = 28 #length of the sliding window
device  = find_device()

# Loading trained LSTM models
lstm_model_filename = os.path.join(MODEL_PATH, '1layer_lstm_model.pth')

loaded_model_info = torch.load(lstm_model_filename)
lstm = LSTM(loaded_model_info['num_classes'],
                   loaded_model_info['input_size'],
                   loaded_model_info['hidden_size'],
                   loaded_model_info['num_layers'])
lstm.load_state_dict(loaded_model_info['model_state_dict'])
lstm.to(device)                                         # Make sure to send the model to the appropriate device
#lstm.eval()

Using MPS device


/Users/nairabarseghyan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


LSTM(
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(1, 512, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [3]:
lstm.summary()

AttributeError: 'LSTM' object has no attribute 'summary'

In [5]:
!pip install torchsummary

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
from torchsummary import summary

In [7]:
print(lstm)

LSTM(
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(1, 512, batch_first=True, dropout=0.25)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)


In [10]:
summary(loaded_model_info, loaded_model_info['input_size'])

TypeError: 'int' object is not iterable

In [ ]:
device = find_device

In [19]:
summary(lstm, input_size=(1,1))

RuntimeError: Input and hidden tensors are not at the same device, found input tensor at cpu and hidden tensor at mps:0